### Question 1

As part of their loyalty reward program, Galeries Lafayette would like to have the list of userid of all users who spent at least three times at Galeries Lafayette. 

How would you obtain this **using only SQL**?

In [1]:
import pandas as pd
import pandasql as psql
from datetime import date, timedelta

In [2]:
# check the dataset and put it in dataframes
date_cols = ['registeredAt', 'firstBankConnectedAt']
df_users = pd.read_csv('users.csv', parse_dates=date_cols)
df_users['age'] = df_users['age'].fillna(0).astype(int)
df_users.head()

,userId,age,gender,registeredAt,firstBankConnectedAt
0,1092,38,F,2019-12-13 09:35:16,2020-01-17 14:12:06
1,1102,24,M,2020-11-20 07:56:03,2020-11-20 07:59:44
2,2989,21,F,2021-01-09 06:44:54,2021-01-09 06:51:54
3,1392,23,M,2019-10-13 21:15:16,2019-10-13 21:18:55
4,3444,29,M,2020-08-07 10:05:23,2020-08-07 10:08:13


In [3]:
df_transactions = pd.DataFrame(pd.read_csv ('transactions.csv'))
df_transactions.head()

,userId,date,merchantId,amountEuros,wording,eligibleForOffer,usedOffer
0,1,2021-09-30,galeries-lafayette,23.5,PAIEMENT PAR CARTE GL BORDEAUX XX/XX,0,0
1,2,2021-09-28,galeries-lafayette,5.9,PAIEMENT PAR CARTE GL SAINT LAURENT XX/XX,0,0
2,3,2021-09-30,galeries-lafayette,250.0,PAIEMENT PAR CARTE GL MONTPELLIERS XX/XX,0,0
3,4,2021-09-28,galeries-lafayette,35.9,CARTE XXXXX XX/XX GL TOULOUSE XXXXXXXXXXXXXXXIOPD,0,0
4,5,2021-09-29,galeries-lafayette,24.0,PAIEMENT PAR CARTE GL NANTES XX/XX,0,0


### SQL query structure:
` SELECT userId, COUNT(*) AS count 
 FROM df_transactions 
 WHERE merchantID='galeries-lafayette' 
 GROUP BY userID 
 HAVING COUNT( 'userID' ) >= 3`

In [4]:
# use pandasql to test the query using dataframes
df_question1 =  psql.sqldf("SELECT userId, COUNT(*) AS count \
                            FROM df_transactions \
                            WHERE merchantID='galeries-lafayette' \
                            GROUP BY userID \
                            HAVING COUNT( 'userID' ) >= 3")
df_question1.head()

,userId,count
0,8,3
1,21,3
2,33,3
3,42,6
4,43,3


Since it is my first time with SQL queries I decided to check if the result is correct using dataframe functions.

In [5]:
# select only galeries-lafayette as merchant 
select_gl = df_transactions[df_transactions.merchantId == 'galeries-lafayette']
# create a new dataframe containing userId and counts
df_test1 = pd.DataFrame({'userId':select_gl['userId'].value_counts().index, 'count':select_gl['userId'].value_counts().values})
# sort the dataframe by ascending userId
df_test1.sort_values(by="userId", inplace=True)
# select userId with at least three purchases at galerie-lafayette
df_test1 = df_test1.loc[df_test1['count'] >=3 ]
df_test1= df_test1.reset_index(drop=True)
df_test1.head()

,userId,count
0,8,3
1,21,3
2,33,3
3,42,6
4,43,3


In [6]:
print("Is the Dataframe obtained with the SQL query correct? ",df_test1.equals(df_question1))

Is the Dataframe obtained with the SQL query correct?  True


### **Question 2**

To better understand the consumption habits of our users, the B2B team would like to identify the users who are customers at Printemps but not at Galeries Lafayette.

Among these users, we are especially interested in the users who have high average basket at Printemps (i.e. above €150)

How would you obtain for this population the list of userid and their average baskets at Printemps **using only SQL**?

*The average basket can be computed as the average of amountEuros when a user has at least one transaction at Printemps.*

### SQL query structure:
` SELECT userId, AVG(amountEuros) AS average_basket
  FROM df_transactions 
  WHERE merchantId='printemps' 
  AND userId NOT IN 
       (SELECT userId FROM df_transactions WHERE merchantId ='galeries-lafayette') 
  GROUP BY userId 
  HAVING average_basket > 150.0`

In [7]:
# use pandasql to test the query using dataframes
df_question2 =  psql.sqldf("SELECT userId, AVG(amountEuros) AS average_basket\
                            FROM df_transactions \
                            WHERE merchantId='printemps' \
                            AND userId NOT IN \
                               (SELECT userId FROM df_transactions WHERE merchantId ='galeries-lafayette') \
                            GROUP BY userId \
                            HAVING average_basket > 150.0")
df_question2.head()

,userId,average_basket
0,2350,165.000
1,2355,219.300
2,2356,295.000
3,2357,170.835
4,2361,179.000


Again I did a little test to reproduce the results with only pandas funcions and check if the results are correct.

In [8]:
# select only printemps as merchant excluding galeries-lafayette
list_userId_gl = select_gl.userId.tolist()
select_pr = df_transactions[(df_transactions.merchantId == 'printemps') ]
select_pr = select_pr[~select_pr['userId'].isin(list_userId_gl)]
# compute the average basket 
select_pr = select_pr.groupby(["userId"], as_index=False).mean()
df_test2 = select_pr[['userId', "amountEuros"]].copy().rename(columns={'amountEuros': 'average_basket'})
# select users who have high average basket at Printemps > 150 €
df_test2 = df_test2.loc[df_test2['average_basket'] > 150.0 ]
df_test2= df_test2.reset_index(drop=True)
df_test2.head()

,userId,average_basket
0,2350,165.000
1,2355,219.300
2,2356,295.000
3,2357,170.835
4,2361,179.000


In [9]:
print("Is the Dataframe obtained with the SQL query correct? ",df_test2.equals(df_question2))

Is the Dataframe obtained with the SQL query correct?  True


### Question 4

The catch-rate is an important metric for the B2B team. 

In business terms, the catch-rate is the proportion of Joko's potential revenue that is actually captured. Here the catch-rate is computed on the amount spent and not on the commission (which is equivalent to Joko's revenue) but if we assume that the commission is a fixed percentage of the amount spent, the output is the same.

The formula to compute the catch-rate is the following:

$$
catchRate=\frac{amountEuros\text{(for transactions having }  {usedOffer}
\text{=1)}}{amountEuros\text{(for transactions having } {eligibleForOffer}
\text{=1)}}
$$

Using **python** (pandas library), please compute the daily / weekly / monthly catch-rate for Galeries Lafayette.

In [10]:
# read th cvs file with transactions with date column as datetime type 
dateparser = lambda x: pd.datetime.strptime(x, "%Y-%m-%d")
df = pd.read_csv("transactions.csv", parse_dates=['date'], date_parser=dateparser)
# select only galeries-lafayette as merchant 
df = df[df.merchantId == 'galeries-lafayette']
df = df.sort_values(by="date")

total_transactions = len(df)
print('Total number of transactions at Galeries Lafayette:', total_transactions)
df.head()

Total number of transactions at Galeries Lafayette: 3100


,userId,date,merchantId,amountEuros,wording,eligibleForOffer,usedOffer
1246,990,2021-06-01,galeries-lafayette,197.9,PAIEMENT PAR CARTE GL AVIGNON XX/XX,0,0
1111,889,2021-06-02,galeries-lafayette,295.0,FACTURE CARTE DU XXXXXX GLAFAYETTE.COM CARTE X...,0,0
1982,1564,2021-06-02,galeries-lafayette,93.0,ACHAT CB GL GRENOBLE XX.XX.XX CARTE NUMERO XXX,0,0
37,33,2021-06-02,galeries-lafayette,115.0,PAIEMENT PAR CARTE GL TOURS XX/XX,0,0
912,736,2021-06-02,galeries-lafayette,100.0,PAIEMENT PAR CARTE GALERIES LAFAYET CHALON SU ...,0,0


The variable catchRate can be computed only if the trnsaction is at least eligible for offers. We can drop the non eligible transactions.

In [11]:
df = df.drop(df[df.eligibleForOffer == 0].index)

total_transactions_eligible = len(df)
print('Total number of transactions at Galeries Lafayette with eligible offers:', total_transactions_eligible)
total_transactions_usedoffer = df.userId[df['usedOffer'] == 1].count()
print('Total number of transactions at Galeries Lafayette with used offers:', total_transactions_usedoffer)
df.head()

Total number of transactions at Galeries Lafayette with eligible offers: 335
Total number of transactions at Galeries Lafayette with used offers: 81


,userId,date,merchantId,amountEuros,wording,eligibleForOffer,usedOffer
1162,395,2021-06-04,galeries-lafayette,85.0,ACHAT CB GL REIMS XX.XX.XX CARTE NUMERO XXX,1,0
1051,847,2021-06-04,galeries-lafayette,89.0,XXXXXX CB****XXXX GL HAUSSMANN XXPARISXXXXXXX/,1,0
1050,846,2021-06-04,galeries-lafayette,2221.0,GL HAUSSMANN,1,0
1153,922,2021-06-04,galeries-lafayette,149.9,PAIEMENT PAR CARTE GL TOULOUSE XX/XX,1,0
1155,922,2021-06-04,galeries-lafayette,129.8,PAIEMENT PAR CARTE GL TOULOUSE XX/XX,1,0


In [12]:
# compute the daily catchRate:
# denominator
den = df.groupby(df['date'].dt.date)['amountEuros'].sum()
# numerator
num = df[df['usedOffer'] == 1].groupby(df['date'].dt.date)['amountEuros'].sum()
# fill daily catchrate in dataframe
df_catchrate_day_frame = {'date': num.divide(den, fill_value=0).index, 'catchRate': num.divide(den, fill_value=0) }
df_catchrate_day = pd.DataFrame(df_catchrate_day_frame).reset_index(drop=True)
df_catchrate_day.head()

,date,catchRate
0,2021-06-04,0.025913
1,2021-06-05,0.122840
2,2021-06-06,0.000000
3,2021-06-07,0.506276
4,2021-06-08,0.000000


In [13]:
# compute the weekly catchRate:
# denominator
den = df.groupby(df['date'].dt.week)['amountEuros'].sum()
# numerator
num = df[df['usedOffer'] == 1].groupby(df['date'].dt.week)['amountEuros'].sum()
# fill daily catchrate in dataframe
df_catchrate_week_frame = {'week': num.divide(den, fill_value=0).index, 'catchRate': num.divide(den, fill_value=0) }
df_catchrate_week = pd.DataFrame(df_catchrate_week_frame).reset_index(drop=True)
df_catchrate_week.head()

,week,catchRate
0,22,0.052403
1,23,0.408410
2,24,0.271091
3,27,0.509383
4,28,0.165018


In [14]:
# compute the monthly catchRate:
# denominator
den = df.groupby(df['date'].dt.month)['amountEuros'].sum()
# numerator
num = df[df['usedOffer'] == 1].groupby(df['date'].dt.month)['amountEuros'].sum()
# fill daily catchrate in dataframe
df_catchrate_month_frame = {'month': num.divide(den, fill_value=0).index, 'catchRate': num.divide(den, fill_value=0) }
df_catchrate_month = pd.DataFrame(df_catchrate_month_frame).reset_index(drop=True)
df_catchrate_month

,month,catchRate
0,6,0.222433
1,7,0.192409
2,8,0.055199
3,9,0.225628
4,10,0.285927
5,11,0.464233


In [15]:
# compute the yearly catchRate:
# denominator
den = df.groupby(df['date'].dt.year)['amountEuros'].sum()
# numerator
num = df[df['usedOffer'] == 1].groupby(df['date'].dt.year)['amountEuros'].sum()
# fill daily catchrate in dataframe
df_catchrate_year_frame = {'year': num.divide(den, fill_value=0).index, 'catchRate': num.divide(den, fill_value=0) }
df_catchrate_year = pd.DataFrame(df_catchrate_year_frame).reset_index(drop=True)
df_catchrate_year

,year,catchRate
0,2021,0.250887


#### From a business perspective, what could be the levers to improve the catch-rate?

Only a limited fraction of transactions are eligibles for offers. In order to improve the catch rate the number of eligible transactions should be firstly expanded. 

In [16]:
print("Percentage of transactions with eligible offer with respect to the total: ", 
      round(total_transactions_eligible/total_transactions*100,1), "%")

Percentage of transactions with eligible offer with respect to the total:  10.8 %


Moreover there is no evident reason why an user that can make a transaction with an offer applied decided not to take advantage of it. Is the offer sufficently visible? Does the user know of it? It could be a visibility problem. To solve this the default settings of the app could be modified in order to have 'Active' the application of an offer on a given transaction. It is however necessary still give the choice to the users to deactivate it. 

In [17]:
print("Percentage of transactions with used offer with respect to the eligible ones: ", 
      round(total_transactions_usedoffer/total_transactions_eligible*100,1), "%")

Percentage of transactions with used offer with respect to the eligible ones:  24.2 %


### Preparaton of the csv file for Question 3

Make a unique file containing the information to study the profiles and consumption habits of the users who are customers at Galeries Lafayette. 
- select users at Galeries Lafayette
- select variables useful for the analysis both from transactions.csv and usrs.csv
- create a unique csv file

In [18]:
# from csv file: transactions
# select only galeries-lafayette as merchant 
users_gl = df_transactions[df_transactions.merchantId == 'galeries-lafayette']

# create a new dataframe containing useful variables: userId, number of purchases, average expenditure, fraction of times the eligible offer was used
df_to_merge1 = pd.DataFrame({'userId':users_gl['userId'].value_counts().index, 
                         'num_of_puchases':users_gl['userId'].value_counts().values,
                         'average_expenditure [€]':round(users_gl.groupby(["userId"])[ 'amountEuros'].mean(),2),
                         'fraction_of_offers_used':(users_gl.groupby(["userId"])['usedOffer'].sum()/users_gl.groupby(["userId"])['eligibleForOffer'].sum()).fillna(-1).astype(int)
                         })\
                        .reset_index(drop=True)
# replaced NaN with -1 just because it is more simple
df_to_merge1 = df_to_merge1.sort_values(by="userId")
df_to_merge1= df_to_merge1.reset_index(drop=True)
df_to_merge1.head()

,userId,num_of_puchases,average_expenditure [€],fraction_of_offers_used
0,1,1,147.45,0
1,2,2,19.85,-1
2,3,2,207.00,-1
3,4,2,41.10,-1
4,5,1,34.00,-1


In [19]:
# from csv file: users
# select only users with transactions at Galeries Lafayette
list_userId_gl = df_to_merge1.userId.tolist()
df_to_merge2 = df_users[df_users['userId'].isin(list_userId_gl)]
# I decided not to use directly the variables 'firstBankConnectedAt' and 'registeredAt'
# a more complete study could make use of them but for simplicity I limit the choice of variables
# instead I use the time difference between the registration and the connection of the bank account
df_to_merge2['timediff_registration_bankconnected [h]'] = round((df_to_merge2['firstBankConnectedAt'] - df_to_merge2['registeredAt']).dt.seconds/3600,3)
# and the date of registration
df_to_merge2['registration_date'] = df_to_merge2['registeredAt'].dt.date
df_to_merge2 = df_to_merge2.drop(['firstBankConnectedAt', 'registeredAt'], 1)
df_to_merge2 = df_to_merge2.sort_values(by="userId")
df_to_merge2= df_to_merge2.reset_index(drop=True)
df_to_merge2.head()

# there is a warning but everything works fine, so no worries

/home/alice/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/alice/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.


,userId,age,gender,timediff_registration_bankconnected [h],registration_date
0,1,20,M,0.086,2020-01-02
1,2,23,F,0.163,2019-08-22
2,3,23,M,0.090,2019-10-14
3,4,37,F,0.113,2020-09-23
4,5,23,F,0.064,2020-01-05


In [20]:
# merge the two dataframes and output a cvs file to be used in the study
df_final= df_to_merge1.merge(df_to_merge2, how='inner')
df_final.head()

,userId,num_of_puchases,average_expenditure [€],fraction_of_offers_used,age,gender,timediff_registration_bankconnected [h],registration_date
0,1,1,147.45,0,20,M,0.086,2020-01-02
1,2,2,19.85,-1,23,F,0.163,2019-08-22
2,3,2,207.00,-1,23,M,0.090,2019-10-14
3,4,2,41.10,-1,37,F,0.113,2020-09-23
4,5,1,34.00,-1,23,F,0.064,2020-01-05


In [21]:
df_final.to_csv(r'users_summary.csv', index = False)

In [22]:
# save as html
!jupyter nbconvert --to html DataAnalyst_CaseStudy.ipynb

[NbConvertApp] Converting notebook DataAnalyst_CaseStudy.ipynb to html
[NbConvertApp] Writing 344706 bytes to DataAnalyst_CaseStudy.html
